In [ ]:
from graph_cnn.model import create_model
from graph_cnn.graph import create_final_graph,cross_over
import networkx as nx
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from IPython.display import clear_output

In [ ]:
graph_ = create_final_graph(5,0.7)
print(graph_)
nx.draw(graph_,with_labels=True)
plt.show()
model = create_model(graph_,input_shape=(28,28,1),num_classes=10)
model.summary()

In [ ]:
train_dataset, info = tfds.load('mnist', split='train', with_info=True, as_supervised=True, shuffle_files=True)
test_dataset = tfds.load('mnist', split='test', as_supervised=True, shuffle_files=True)

# Print dataset information
print(info)

train_dataset = train_dataset.cache().shuffle(1000).batch(128).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).batch(128).prefetch(tf.data.experimental.AUTOTUNE)
for i,j in train_dataset:
    print(i)

In [ ]:
input_layer = tf.keras.layers.Input(shape=(28, 28, 1))
#random_flip = tf.keras.layers.RandomFlip("horizontal")(input_layer)
#random_rotation = tf.keras.layers.RandomRotation(0.2)(random_flip)
#random_zoom = tf.keras.layers.RandomZoom(0.2)(random_rotation)
#gaussian_noise = tf.keras.layers.GaussianNoise(0.3)(random_zoom)
rescaling = tf.keras.layers.Rescaling(1./255)(input_layer)
output = model(rescaling)

model2 = tf.keras.models.Model(inputs=input_layer, outputs=output)


In [ ]:
METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.FalseNegatives(name='fn'),
      #tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=METRICS)
model2.summary()
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=10,min_delta=0.01, min_lr=0)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        y_pred =  np.array(self.model.predict(test_dataset))[0]
        predicted_labels = np.argmax(y_pred, axis=1)
        true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
        cm = confusion_matrix(true_labels, predicted_labels)
        print(cm)

In [ ]:
tf.keras.utils.plot_model(model2, to_file="model.png", show_shapes=True, show_layer_names=True, rankdir="TB", expand_nested=True, dpi=96)

In [ ]:
'''model2=tf.keras.applications.densenet.DenseNet121(include_top=False, weights=None, input_shape=(36,36,3), pooling=None, classes=10)
model = tf.keras.models.Sequential([
    model2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()'''

In [ ]:
# Predict on the test dataset
predictions = model2.predict(test_dataset)
predictions = np.array(predictions)[0]
# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels
true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
print(true_labels,predicted_labels.shape)
print(true_labels.shape,predicted_labels.shape)

# Plot the classification results
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print(cm)

In [10]:
model2.fit(train_dataset, epochs=100, validation_data=test_dataset,callbacks=[reduce_lr,early_stop,DisplayCallback()])

79/79 [==============================] - 28s 357ms/step
[[  0 496   4 201  29   0 173  76   0   1]
 [  0 563   4 238  31   0 216  80   0   3]
 [  0 501   5 211  33   0 180  96   2   4]
 [  0 519   3 197  27   0 177  83   1   3]
 [  0 510   3 188  30   0 173  75   1   2]
 [  0 455   2 197  17   0 163  57   0   1]
 [  0 487   5 182  16   0 180  79   1   8]
 [  0 501   8 212  20   0 202  81   0   4]
 [  0 502   4 200  26   0 174  65   0   3]
 [  0 529   7 200  21   1 172  76   2   1]]
469/469 [==============================] - 651s 1s/step - loss: 3.9283 - model_loss: 3.0813 - model_1_loss: 0.7103 - model_2_loss: 0.1368 - model_accuracy: 0.0939 - model_1_accuracy: 0.1046 - model_2_accuracy: 0.0991 - val_loss: 5.0636 - val_model_loss: 2.2881 - val_model_1_loss: 2.1962 - val_model_2_loss: 0.5793 - val_model_accuracy: 0.0000e+00 - val_model_1_accuracy: 0.0661 - val_model_2_accuracy: 0.1010 - lr: 1.0000e-04
Epoch 6/100
113/469 [======>.......................] - ETA: 7:23 - loss: 3.5547 - mode